In [1]:
import pandas as pd
import numpy as np
import nltk
import re

In [2]:
import nltk
nltk.__version__



'3.8.1'

In [3]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>


False

In [4]:
# df = pd.read_csv("training.csv", encoding="ISO-8859-1")
df = pd.read_csv("training.csv", header=None, names=["target", "unique_id", "date", "flag", "user_id", "text"], encoding="ISO-8859-1")
df.sample(10)

,target,unique_id,date,flag,user_id,text
1117323,4,1973400238,Sat May 30 11:04:29 PDT 2009,NO_QUERY,rdy2freefall,@theresanoelle hahaha yup you fail three days ...
112553,0,1825485684,Sun May 17 05:52:12 PDT 2009,NO_QUERY,Jonloge,My mom DOES NOT know how to walk in the city! ...
710483,0,2257923540,Sat Jun 20 15:06:37 PDT 2009,NO_QUERY,shariselw,"im tired of these hussie (@BscoTT26, @LuvNless..."
1315995,4,2014101472,Wed Jun 03 00:27:52 PDT 2009,NO_QUERY,Ginnka,"@spoof Ð½Ñ Ð²Ð¾Ñ, ÑÐµÐ¿ÐµÑÑ Ñ? ÑÐ°Ñ?Ñ?Ñ..."
341833,0,2015132761,Wed Jun 03 03:48:08 PDT 2009,NO_QUERY,Lanaaa93,"@NicolePaige oooh it will get better, I will ..."
620181,0,2228243059,Thu Jun 18 14:11:46 PDT 2009,NO_QUERY,allyycase,Mah face is sunburnt. Brings out my freckles ...
829271,4,1557149183,Sun Apr 19 02:17:46 PDT 2009,NO_QUERY,i_am_fabs,"What Mankind Can Achieve in 40 years, http://d..."
460983,0,2169495080,Sun Jun 14 14:35:51 PDT 2009,NO_QUERY,roxiijonas,At first no one had commented JBs pic i was ne...
1562263,4,2186788690,Mon Jun 15 19:20:19 PDT 2009,NO_QUERY,ebarlean21,Painting a wall in my apartment #96163 Bavaria...
1328009,4,2015462774,Wed Jun 03 04:43:49 PDT 2009,NO_QUERY,stockstef,Preparing for my sisters wedding on friday and...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   target     1600000 non-null  int64 
 1   unique_id  1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   flag       1600000 non-null  object
 4   user_id    1600000 non-null  object
 5   text       1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [6]:
df.shape

(1600000, 6)

In [7]:
df.drop(['unique_id','date','flag','user_id'], axis = 1, inplace = True) 

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   text    1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [9]:
df.sample(10)

,target,text
480178,0,didn't they say they'd edit it!!!!
154939,0,am going to school: hate it
1416978,4,Wanting to see baby Alysia. she brightens up ...
1478473,4,@silverraindog Guess thats what cousins are! T...
964052,4,@cindygarman I think it works the other way ar...
636332,0,"@mokudekiru Yay! Also, I tried to text you, bu..."
1067822,4,@mitchelmusso Im waiting on you to answer :'[ ...
303340,0,@Andiie15 always the same who as all the thing...
962739,4,@KY502NTX @Astro_Mike is currently spacewalkin...
1591434,4,@lightscameraash GOOD LUCK.


In [10]:
df.rename(columns={'target': 'sentiment', 'text': 'tweet'}, inplace = True)

In [11]:
df.sample(10)

,sentiment,tweet
683807,0,Too much drinking makes my mouth num I'm num....
909282,4,Happy Mother's Day! She liked the poem.
824295,4,@FADIAKADER I'm here for a MIN! This friday is...
1197456,4,@socrazygirl_ it's perfectly fine as long as y...
1097330,4,@CarbonoxRatchet you probably typed a word tha...
89525,0,I feel like the worst daughter ever. I didn't ...
968587,4,Just finished one of Sunday afternoon's most c...
1537180,4,@trainbetter: how are you studying for the csc...
1422548,4,D-Day Jay-Z Palm Pre Tetris Belmont Stakes Lau...
134171,0,@Icy1_Female Tell me about - I swear I'm still...


In [12]:
df = df.dropna()
df.sample(10)

,sentiment,tweet
1350935,4,fallin asleep at the computer....come on ben l...
412956,0,just figured out that I have not been receivin...
1487937,4,i love you much its crazy
1223182,4,im following 666 people thet explein's alot a ...
175097,0,"i donbt like to peel prawns, i also dont like ..."
137007,0,working on product catalogue.. the program sud...
599388,0,Been staying back in the office for the longes...
1019577,4,I need a good day sailing. Thank lord its frid...
757605,0,The passenger was one of my Bounty friends. T...
333601,0,Sounds like Air France disaster is as bad as t...


In [13]:
print(df.info())
print("The Shape Of The Dataset Is :",df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   tweet      1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB
None
The Shape Of The Dataset Is : (1600000, 2)


In [14]:
df['sentiment'] = df['sentiment'].replace({0: 'negative', 4: 'positive'})

In [15]:
df.sample(5)

,sentiment,tweet
366920,negative,"Last day of school!!! I felt so happy, but all..."
1387213,positive,Is about to start some HARDORE revising
1064636,positive,@katiejoseph1 thanks so much so i'm thinking ...
643875,negative,My Router is playing silly buggers with AIM ag...
486908,negative,@artistikem it was a napoleon dynamite referen...


In [16]:
# Remove URLs, special characters, punctuation, and numbers from tweet column
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub('[^a-zA-Z\s]', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub('[0-9]+', '', x))


In [17]:
print(df.sample(10))
print('\033[1m' + 'The information of the dataset is :' + '\033[0m')
print(df.info())
print('\033[1m' + 'The Shape Of The Dataset Is :' + '\033[0m',df.shape)

        sentiment                                              tweet
383216   negative  pmam relay for life on a cold nightNOT A GOOD ...
1183527  positive  therealsavannah i love it when you do covers o...
31160    negative       gfalcone Im not going to the tour at all  xx
806464   positive    When Chelsea brings me chocolate and diet co...
1476993  positive                            looking forward to tea 
911793   positive     lolmy neighbors must think im nutsfor canucks 
1079972  positive  NewRule If you cant spell ityou might not want...
1415383  positive  now im really ready to get my day started SHE ...
993444   positive  coldplay good night guy all the way from perth...
1276324  positive  goodsex is the best way to get you through out...
The information of the dataset is :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentim

In [18]:

df['tweet'] = df['tweet'].str.lower()

In [19]:
stop_words = set(stopwords.words('english'))
df['tweet'] = df['tweet'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))


NameError: name 'stopwords' is not defined